In [1]:
import pandas as pd

movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [2]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [3]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
len(movies.title.unique())

9737

In [5]:
movies[movies.duplicated(['title'], keep=False)]

,movieId,title,genres
650,838,Emma (1996),Comedy|Drama|Romance
2141,2851,Saturn 3 (1980),Adventure|Sci-Fi|Thriller
4169,6003,Confessions of a Dangerous Mind (2002),Comedy|Crime|Drama|Thriller
5601,26958,Emma (1996),Romance
5854,32600,Eros (2004),Drama
5931,34048,War of the Worlds (2005),Action|Adventure|Sci-Fi|Thriller
6932,64997,War of the Worlds (2005),Action|Sci-Fi
9106,144606,Confessions of a Dangerous Mind (2002),Comedy|Crime|Drama|Romance|Thriller
9135,147002,Eros (2004),Drama|Romance
9468,168358,Saturn 3 (1980),Sci-Fi|Thriller


Found some duplicate data

In [6]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [7]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [8]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from scrapy.exceptions import DropItem
from models import Movie, MovieGenre, MovieRating, create_table

engine = create_engine('sqlite:///movies.db')
create_table(engine)
Session = sessionmaker(bind=engine)

- for each movie in csv file find movie in database and store the csv files movieId
- add the genres to the movie in database
- add the ratings using the stored movieId

In [10]:
import datetime

session = Session()
for index, row in movies[:100].iterrows():
    # print(row['title'])
    # title, year = row['title'].replace(')', '').split(' (')
    # print(title, year)
    # print(row['title'][-7:])
    title = row['title'][:-7]
    year = row['title'][-7:].replace(' (', '').replace(')', '')
    # print(year)

    movie = session.query(Movie).filter_by(
        name = title, year = year
    ).first()

    if movie is not None:
        # print(movie.name, movie.id)
        genres = row['genres'].split('|')
        # print(genres)

        for genre in genres:
            movie.genre.append(MovieGenre(genre = genre))
        filtered_ratings = ratings[ratings['movieId'] == row['movieId']]
        for index, rating_row in filtered_ratings.iterrows():
            movie.rating.append(MovieRating(user_id = rating_row['userId'], rating = rating_row['rating'], timestamp = datetime.datetime.fromtimestamp(rating_row['timestamp'])))


        session.add(movie)
        session.commit()

